In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/Train.csv')
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
train.isnull().sum()/len(train)*100

Item_Identifier               0.000000
Item_Weight                  17.165317
Item_Fat_Content              0.000000
Item_Visibility               0.000000
Item_Type                     0.000000
Item_MRP                      0.000000
Outlet_Identifier             0.000000
Outlet_Establishment_Year     0.000000
Outlet_Size                  28.276428
Outlet_Location_Type          0.000000
Outlet_Type                   0.000000
Item_Outlet_Sales             0.000000
dtype: float64

In [ ]:
train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0], inplace=True)
train['Item_Weight'].fillna(train['Item_Weight'].mean(), inplace=True)

In [ ]:
train.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [ ]:
train = pd.get_dummies(train, columns = ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],prefix = '',prefix_sep = '')

In [ ]:
train = pd.get_dummies(train, columns = ['Outlet_Identifier'],prefix = '',prefix_sep = '')

In [ ]:
train.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,LF,Low Fat,Regular,low fat,...,OUT010,OUT013,OUT017,OUT018,OUT019,OUT027,OUT035,OUT045,OUT046,OUT049
0,FDA15,9.30,0.016047,249.8092,1999,3735.1380,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,48.2692,2009,443.4228,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,141.6180,1999,2097.2700,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.000000,182.0950,1998,732.3800,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.000000,53.8614,1987,994.7052,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
for i in train.columns[1:]:
    train[i] = (train[i] - train[i].min()) / (train[i].max() - train[i].min())

In [ ]:
train = train.drop('Item_Identifier', axis=1)

In [ ]:
X = train.drop('Item_Outlet_Sales', axis=1)

y = train['Item_Outlet_Sales']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=10,test_size=0.2)

In [ ]:
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((6818, 45), (6818,)), ((1705, 45), (1705,)))

In [ ]:
import keras
print(keras.__version__)

2.12.0


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
from keras.models import Sequential

In [ ]:
from keras.layers import InputLayer, Dense,Flatten,Dropout

In [ ]:
X_train.shape

(6818, 45)

In [ ]:
X_train.shape[1]

45

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(128,kernel_initializer='normal',input_shape=(X_train.shape[1],),activation='relu'))

In [ ]:
model.add(Dense(256,kernel_initializer='normal',activation='relu'))
model.add(Dense(256,kernel_initializer='normal',activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256,kernel_initializer='normal',activation='relu'))

In [ ]:
model.add(Dense(1,kernel_initializer='normal',activation='linear'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5888      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 170,753
Trainable params: 170,753
Non-trai

In [ ]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mean_absolute_error'])

In [ ]:
model_history = model.fit(X_train,y_train,epochs=300,batch_size=60,validation_split=0.2)

Epoch 1/300
91/91 [==============================] - 3s 17ms/step - loss: 0.0727 - mean_absolute_error: 0.0727 - val_loss: 0.0588 - val_mean_absolute_error: 0.0588
Epoch 2/300
91/91 [==============================] - 1s 12ms/step - loss: 0.0614 - mean_absolute_error: 0.0614 - val_loss: 0.0580 - val_mean_absolute_error: 0.0580
Epoch 3/300
91/91 [==============================] - 1s 12ms/step - loss: 0.0611 - mean_absolute_error: 0.0611 - val_loss: 0.0584 - val_mean_absolute_error: 0.0584
Epoch 4/300
91/91 [==============================] - 1s 6ms/step - loss: 0.0600 - mean_absolute_error: 0.0600 - val_loss: 0.0587 - val_mean_absolute_error: 0.0587
Epoch 5/300
91/91 [==============================] - 1s 7ms/step - loss: 0.0592 - mean_absolute_error: 0.0592 - val_loss: 0.0583 - val_mean_absolute_error: 0.0583
Epoch 6/300
91/91 [==============================] - 1s 7ms/step - loss: 0.0595 - mean_absolute_error: 0.0595 - val_loss: 0.0592 - val_mean_absolute_error: 0.0592
Epoch 7/300
91/91 [

In [ ]:
prediction = model.predict(X_test)

54/54 [==============================] - 0s 2ms/step


In [ ]:
prediction

array([[0.2750923 ],
       [0.11984784],
       [0.1351529 ],
       ...,
       [0.0535614 ],
       [0.26889634],
       [0.07705434]], dtype=float32)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

mse = mean_squared_error(y_test, prediction)
rmse = mean_squared_error(y_test, prediction, squared=False)
mae = mean_absolute_error(y_test, prediction)
mdae = median_absolute_error(y_test, prediction)
r2 = r2_score(y_test, prediction)

print('The evaluation scores for the ANN model are:')
print('R2:', r2)
print('MSE:', mse)
print('RMSE:', rmse)
print('MAE:', mae)
print('MDAE:', mdae)

The evaluation scores for the ANN model are:
R2: 0.3381651964239938
MSE: 0.01152943890524462
RMSE: 0.10737522482046136
MAE: 0.07606321682762442
MDAE: 0.05088797999646821
